In [1]:
import math
import random
import pandas as pd
import matplotlib.pyplot as plt
from statistics import NormalDist as normal
from scipy.stats import norm
from numpy import log as ln
from tabulate import tabulate

#########
##MINGZE @2022
#########

In [2]:
#prices calls and puts using BS formula

def d1f(s,k,r,t,vol):
    d1 = (ln(s/k)+(r+vol*vol/2)*t)/(vol*math.sqrt(t));
    return d1;

def d2f(s,k,r,t,vol):
    d1 = d1f(s,k,r,t,vol);
    d2 = d1 - vol*math.sqrt(t);
    return d2;

def call_pricing(s,k,r,t,vol):
    c = norm.cdf(x = d1f(s,k,r,t,vol))*s - norm.cdf(x = d2f(s,k,r,t,vol))*k*math.exp(-r*t);
    return c
    
def put_pricing(s,k,r,t,vol):
    p = norm.cdf(x = -d2f(s,k,r,t,vol))*k*math.exp(-r*t) - norm.cdf(x = -d1f(s,k,r,t,vol))*s; 
    return 

In [3]:
def create_MC(vol, maturity, stock0=100):
    stock_initial = stock0;
    stock = stock_initial;
    #time series list
    mc_ts = [];
    counter=1;

    for d in range(round(maturity)):
        mc_ts.append(stock);
        rand = random.uniform(0,1);
        change=stock*vol*normal(mu=0, sigma=1).inv_cdf(rand);
        stock+=change;
        counter+=1;
    return mc_ts;

#vol is annualized; m in days
def simulateMC(vol = 0.16, m = 252, s=100, k=100, r=20000):  
    vol_day = vol/16;
    sum = 0;
    for i in range(r):    
        lst = create_MC(vol_day, m, stock0=s);
        sum+=max(lst[round(m)-1]-k,0);
    avg = sum/r;
    return avg;

def simulatePrice(vol = 0.16, m = 252, s=100, r=20000):  
    vol_day = vol/16;
    sum = 0;
    for i in range(r):    
        lst = create_MC(vol_day, m, stock0=s);
        sum+=lst[round(m)-1];
    avgLastPrice = sum/r;
    return avgLastPrice;

In [4]:
def getDelta(vol, maturity, stock=100, strike=120,  rounds=20000):
    s101 = simulateMC(vol, m = 252*maturity, s=stock*1.01, k=strike, r=rounds);
    s100 = simulateMC(vol, m = 252*maturity, s=stock, k=strike, r=rounds);   
    return 100*(s101-s100);

def getGamma(vol, maturity, stock=100, strike=120, rounds=20000):
    d01 = getDelta(vol, maturity, stock*1.01, strike, rounds);
    d00 = getDelta(vol, maturity, stock, strike, rounds);
    return d01-d00;

def getVega(vol, maturity, strike=120, rounds=20000):
    v01 = simulateMC(vol+0.01, m = 252*maturity, s=100, k=strike, r=rounds);
    v00 = simulateMC(vol, m = 252*maturity, s=100, k=strike, r=rounds);
    return v01-v00;

In [5]:
rounds = 20000
vol = 0.16

##using MC approach
getGamma(vol, 1/12, 100, 100, rounds)

7.99815571193399

Cliquet Pricer with Monte Carlo Simulation

In [6]:
import scipy
import math

def cliquet_pricer(s, k1, k2, T, sigma, rounds, restrike):
  vol = sigma/math.sqrt(252)*math.sqrt(252/restrike)   
  sum = 0
  for round in range(rounds):
    s_new = s 
    s_curr = s
    payout = 0
    kk1 = k1
    kk2 = k2
    tenor = int(252/restrike)
    for ith in range(restrike):
      for jth in range(tenor):
          s_new += s_curr * vol * scipy.random.normal()
      payout += min(max(s_new - kk1,0), (kk2 - kk1))
      kk1 = kk1 * (s_new/s_curr)
      kk2 = kk2 * (s_new/s_curr)
      s_curr = s_new
    sum += payout
  return sum/rounds

In [7]:
def cs_pricer(s, k1, k2, T, sigma, rounds):
    return cliquet_pricer(s, k1, k2, T, sigma, rounds, 1)

In [8]:
#Python pricer for cliquet option that restrike every month 
# with strikes 100 and 120, and one year maturity 
print(cliquet_pricer(100, 100, 120, 1, 0.16, 10000, 12))

#Compare the price with that of a 1Y call spread with the same strikes
print(cs_pricer(100, 100, 120, 1, 0.16, 10000))

77.07792741199195
9.889862859609238


Crash Cliquet Pricing with Monte Carlo

In [12]:
import scipy
import math

def crach_cliquet_pricer(s, k1, k2, T, sigma, rounds, restrike):
  vol = sigma/math.sqrt(252)  
  sum = 0
  for round in range(rounds):
    s_new = s 
    s_curr = s
    payout = 0
    kk1 = k1
    kk2 = k2
    for ith in range(restrike):
      #for jth in range(tenor):
      s_new += s_curr * vol * scipy.random.normal() 
      payout += min(max(kk2 - s_new,0), (kk2 - kk1))
      kk1 = kk1 * (s_new/s_curr)
      kk2 = kk2 * (s_new/s_curr)
      s_curr = s_new
    sum += payout
  return sum/rounds

In [16]:
payout = crach_cliquet_pricer(100, 70, 80, 1, 0.16, 6000, 252)
print("price = " + str(price))

price = 0.0


In [15]:
#what are its greeks? 
delta = crach_cliquet_pricer(100*1.01, 70, 80, 1, 0.16, 6000, 252)-crach_cliquet_pricer(100, 70, 80, 1, 0.16, 6000, 252)

delta01=crach_cliquet_pricer(100*1.01*1.01, 70, 80, 1, 0.16, 6000, 252)-crach_cliquet_pricer(100*1.01, 70, 80, 1, 0.16, 6000, 252)

gamma = delta01 - delta

vega = crach_cliquet_pricer(100, 70, 80, 1, 0.16+0.01, 6000, 252) - crach_cliquet_pricer(100, 70, 80, 1, 0.16, 6000, 252)

print("delta = " + str(delta) + "\ngamma = " + str(gamma) + "\nvega = " +str(vega))


delta = 0.0
gamma = 0.0
vega = 0.0
